# Neural Networks

One of the models we chose to implement for the purpose of lung cancer classification was a multilayer perceptron, a type of feedfoward neural network.  
MLPs are suitable for classification prediction problems and data provided in a tabular format, such as the features CSV file we are working with.

We start by importing relevant libraries and dropping useless columns from our CSV.

In [ ]:
from kfold_and_metrics import *

import tensorflow as tf
import pandas as pd
import numpy as np

tf.keras.backend.clear_session()

In [ ]:
df = pd.read_csv("final.csv")
df = df.drop(columns=['id'])
df.head()

### Parameter Hypertuning

In the machine learning world, it is mostly agreed that a single hidden layer is enough to develop a good neural network model.  

Taking this into consideration, we still needed to decide on the number of nodes it should hold, while trying to prevent both under and overfitting. For this, we tested values until 2/3 of the number of input nodes, one of the several rules of thumb for choosing the number of nodes in a hidden layer.  

Additionally, we tested different learning rates for our Stochastic gradient descent optimizer and different activation functions for the hidden layer.

In [ ]:
best_auc = 0
best = {}

# input_nodes = len(df.columns)-2
input_nodes = 50
max_n_nodes = input_nodes * 2//3

for hidden_layer_act in ["softmax", "relu", "sigmoid"]:
    for n_nodes in range(5, max_n_nodes, 10):
        for l_rate in [0.001, 0.003, 0.005, 0.007, 0.01]:
            params = {'hidden_layer_nodes': n_nodes, 'hidden_layer_activation': hidden_layer_act, 'learning_rate': l_rate}
            print("Current parameter combination:")
            for parameter, value in params.items():
                print(f"\t{parameter}: {value}")
            print()

            nn_model = tf.keras.models.Sequential([
                tf.keras.layers.Input((input_nodes,), name="input"),
                tf.keras.layers.Dense(n_nodes,activation=hidden_layer_act),
                tf.keras.layers.Dense(2,activation='softmax')
            ])
            nn_model.compile(
                optimizer=tf.keras.optimizers.SGD(l_rate), 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
            )

            score = k_fold_cv_keras(compiled_model=nn_model, df=df, pca_components=50)
            results = mean_std_results_k_fold_CV(score)

            auc_avg = results.iloc[2,1]
            if auc_avg > best_auc:
                best_auc = auc_avg
                best = params

After running, we got the following values:

In [ ]:
print("Results of the grid search parameter hypertunning:")
for parameter, value in best.items():
    print(f"\t{parameter}: {value}")

The number of hidden layers and rule of thumb considered can be read about in the following resources: